# Library Import and Functions

In [1]:
## Import Libraries
import pandas as pd
import numpy as np
import re

## Display all rows of pandas dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
'''
name: relative_transcript_abundance

purpose: calculate relative transcript abundance

input: a dataframe with a ref_gene_id column identifying the transcript gene of origin and a cov columns with 
the coverage for the transcripts.

output: the same dataframe with median relative abundance and mean gene CPM columns added
'''

def relative_transcript_abundance(df):
    
    first=True
    
    dff = df.copy()

    # for each sample's CPM column, calculate the relative abundance of the transcript in the gene and create a new column with that value
    for col in dff.filter(regex='bam_CPM').columns:
        
        col_gene_name = col.split("_CP")[0] + "_total_gene_CPM"
        col_relative_abundance = col.split("_CP")[0] + "_relative_abundance"
    
        dff_sums = dff[["gene_id", col]].groupby("gene_id").sum()

        dff_sums[col_gene_name] = dff_sums[col].copy()

        dff_sums.drop(columns=col, inplace=True)

        if first:
            merged_dff = pd.merge(dff, dff_sums, how='inner', on="gene_id")
            merged_dff[col_relative_abundance] = ((merged_dff[col]/merged_dff[col_gene_name]) * 100)
            
        else:
            merged_dff = pd.merge(merged_dff, dff_sums, how='inner', on="gene_id")
            merged_dff[col_relative_abundance] = ((merged_dff[col]/merged_dff[col_gene_name]) * 100)
        
        first=False
        
    for col_count in merged_dff.filter(regex='bam_count').columns:
        
        col_gene_name = col_count.split("_count")[0] + "_total_gene_counts"
        
        dff_sums = merged_dff[["gene_id", col_count]].groupby("gene_id").sum()
        dff_sums[col_gene_name] = dff_sums[col_count].copy()
    
        
        dff_sums.drop(columns=col_count, inplace=True)
        
        merged_dff = pd.merge(merged_dff, dff_sums, how='inner', on="gene_id")
        
    merged_dff.fillna(value=0, inplace=True)
    
    # calculate the median relative abundance for all the samples and drop the sample specific relative abundance columns
    rel_ab_col = merged_dff.filter(regex='bam_relative_abundance').columns
    merged_dff['median_relative_abundance'] = merged_dff[rel_ab_col].copy().median(axis=1)
    merged_dff.drop(columns = rel_ab_col, inplace = True)
    
    # calculate the median gene CPM and drop sample specific gene CPM
    gene_cpm_col = merged_dff.filter(regex='bam_total_gene_CPM').columns
    merged_dff['median_gene_cpm'] = merged_dff[gene_cpm_col].copy().median(axis=1)
    merged_dff.drop(columns = gene_cpm_col, inplace = True)
        
    return merged_dff

In [3]:
'''
function name: fix_column_names

purpose: Fixing the column names, making them smaller, informative, and consistent

input: The raw counts dataframe for either genes or transcripts 

output: Same dataframe with improved column names
'''

def fix_column_names(df, is_gene=False):
    
    ## Check if this is a gene counts object
    if is_gene:
        
        ## Get count column names and create list of new column names
        count_columns = df.columns.tolist()
        list_new_names = ["gene_id"]
        
        ## gene_id comes in as index for gene counts data, make it into the first column instead
        df["gene_id"] = df.index
        cols = list(df.columns)
        cols = [cols[-1]] + cols[:-1]
        df = df[cols]
        df.reset_index(inplace=True, drop=True)
    
    ## If it is a transcript dataset
    else:
        ## Set count columns and create list of new names
        count_columns = df.columns[2:].tolist()
        list_new_names = [ "transcript_id", "gene_id"]
    
    ## Fix names one by one and add to list of new names
    for col in count_columns:
        col = col.split("_mapped")[0] + "_counts"
        list_new_names.append(col)
    
    ## Rename columns
    df.columns = list_new_names
    
    return df 

In [4]:
'''
function name: parse_df_columns

purpose: parsing the last aggregate column of the gtf/gff3 into useful columns and cleaning non-relevant columns

input: dataframe containining "raw" gtf/gff

output: dataframe containing gtf with useful columns ["gene_id", "transcript_id", etc...]
'''

def parse_df_columns(df, is_ref=True, is_transcript=False, is_prot=False):

    if is_ref:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split("ne_id \"", expand=True)[1].str.split('\";', expand=True)[0]
        
        ## Get gene names
        df["gene_name"] = df["other"].str.split("gene_name \"", expand=True)[1].str.split('\";', expand=True)[0]
        
        ## Get get transcript biotype
        df["gene_biotype"] = df["other"].str.split('gene_biotype "', expand=True)[1].str.split('"', expand=True)[0]
        
        ## If is transcript get transcript id and transcript biotype
        if is_transcript:
            df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]
            df["transcript_biotype"] = df["other"].str.split('transcript_biotype "', expand=True)[1].str.split('"', expand=True)[0]
            
            ## If is prot get protein_id
            if is_prot:
                df["protein_id"] = df["other"].str.split('protein_id "', expand=True)[1].str.split('"', expand=True)[0]
                df["ccds_id"] = df["other"].str.split('ccds_id "', expand=True)[1].str.split('"', expand=True)[0]
                df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)
        

    else:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)

        ## Get transcript ids
        df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]

        ## Get exon number
        df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)

    for col in df.columns:
        df.loc[df[col].isnull(), col] = np.NaN
        

    return df

In [5]:
'''
function name: calculate_cpm

purpose: Calculate CPM for the each sample given

input: Counts dataset

output: Counts dataset with CPM columns as well
'''

def calculate_cpm(df, is_gene=False):

    ## Set count columns if dataframe is gene counts
    if is_gene:
        count_columns = df.columns[1:].tolist()
    
    ## Set count columns if dataframe is transcript counts
    else:
        count_columns = df.columns[2:].tolist()

    ## Loop through counts columns to calculate CPM and add to the dataframe
    for col in count_columns:
        
        df[col] = round(df[col], 2)
        cpm_name = col.replace("_counts", "_CPM")
        df[cpm_name] = round(((df[col]/(df[col].sum())) * 1000000), 2)
    
    return df  

In [6]:
'''
function name: get_samp_id

purpose: 

input: sample string name

output: sample string name with bam appended
'''


def get_samp_id(s):
    
    name_array = s.split('.')
    last_element = name_array[-1].split('_')[0]
    string_1 = '-'.join(name_array[:-1])
    string_2 = f"{last_element}.bam"
    return f"{string_1}.{string_2}"

In [7]:
## define tissues we are using
tissues_to_use = ["Brain - Cerebellar Hemisphere",
                   "Brain - Frontal Cortex (BA9)",
                   "Brain - Putamen (basal ganglia)",
                   "Cells - Cultured fibroblasts",
                   "Heart - Atrial Appendage",
                   "Heart - Left Ventricle",
                   "Liver",
                   "Lung",
                   "Muscle - Skeletal"]

# Expression of new transcripts since 2019 in our Long-read dataset

## Import 2023 reference and disease gene lists

In [8]:
## Open original reference
ref_2023 = pd.read_csv("../../references/Homo_sapiens.GRCh38.109.gtf", header=None, delimiter="\t", low_memory=False, 
                       names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"], comment="#")

# filter out anything with ERCC (shouldn't be there, but just in case)
ref_2023 = ref_2023.loc[~ref_2023["chr"].str.startswith("ERCC-")]

## Parse through reference to get gene names and ids
gene_ref_2023 = ref_2023.loc[ref_2023["type"]=="gene"].copy()
gene_ref_2023 = parse_df_columns(gene_ref_2023, is_ref=True)
gene_info = gene_ref_2023[['gene_id', 'gene_biotype', 'gene_name']].copy()

## Create 2023 transcript ref
transcript_ref_2023 = ref_2023.loc[ref_2023["type"]=="transcript"].copy()
transcript_ref_2023 = parse_df_columns(transcript_ref_2023, is_ref=True, is_transcript=True)
# get the transcript biotypes
transcript_biotype = transcript_ref_2023[['transcript_id', 'transcript_biotype']].copy()

## Import disease relevant genes
disease_relevant_genes = pd.read_csv("../../references/medically_relevant_genes.tsv", sep="\t")

## Brain disease genes
brain_disease_gene_ids = pd.read_csv("../../references/brain_disease_genes_only_IDs.tsv", sep="\t")
brain_disease_annotations = pd.read_csv("../../references/brain_disease_genes_with_disease.tsv", sep="\t")


## Create disease relevant list including chromosome
disease_relevant_genes_annotated = disease_relevant_genes.merge(gene_ref_2023[["gene_id", "gene_name", "chr"]], 
                                                               how="inner", on=["gene_id", "gene_name"])

## Create list of protein coding genes
protein_coding_ref_2023 = gene_ref_2023.loc[gene_ref_2023["gene_biotype"] == "protein_coding"].copy()

## Import 2019 reference

In [9]:
## Open original reference
ref_2019 = pd.read_csv("../../references/Homo_sapiens.GRCh38.96.gtf", header=None, delimiter="\t", low_memory=False, 
                       names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"], comment="#")

# filter out anything with ERCC (shouldn't be there, but just in case)
ref_2019 = ref_2019.loc[~ref_2019["chr"].str.startswith("ERCC-")]

## Parse through reference to get gene names and ids
gene_ref_2019 = ref_2019.loc[ref_2019["type"]=="gene"].copy()
gene_ref_2019 = parse_df_columns(gene_ref_2019, is_ref=True)
gene_info_2019 = gene_ref_2019[['gene_id', 'gene_biotype', 'gene_name']].copy()

## Create 2023 transcript ref
transcript_ref_2019 = ref_2019.loc[ref_2019["type"]=="transcript"].copy()
transcript_ref_2019 = parse_df_columns(transcript_ref_2019, is_ref=True, is_transcript=True)
transcript_biotype_2019 = transcript_ref_2023[['transcript_id', 'transcript_biotype']].copy()
transcript_ref_2019_ids = transcript_ref_2019["transcript_id"].copy()

## Load reference for protein coding sequences

In [10]:
ref_cds = pd.read_csv("../../tables/transcript_id_and_unique_protein_id_reference.tsv", delimiter="\t", low_memory=False)

## Load transcript counts matrix and CPM normalize it

In [11]:
# samples we are not using due to replicates, low read depth, and poor PCA clustering
filter_out_samples = [ 
    "GTEX-Q2AG-0011-R11A-SM-2EBL2_rep2.FAK44637.bam", 
    "GTEX-Q2AG-0011-R11A-SM-2EBL2_rep.FAK49243.bam", 
    "GTEX-T5JC-0011-R10A-SM-2TT23.FAK91589.bam", 
    "GTEX-QEG5-0008-SM-3QHW2_exp.FAK30166.bam", 
    "GTEX-QV44-0008-SM-3QNG7_ctrl1.FAK55556.bam",
    "GTEX-QV44-0008-SM-3QNG7_exp.FAK52124.bam",
    "GTEX-RWS6-0008-SM-3QHWG_rep.FAK49207.bam", 
    "GTEX-S4Z8-0008-SM-2Y983_exp1.FAK55723.bam",
    "GTEX-S4Z8-0008-SM-2Y983_exp2.FAK47416.bam",
    "GTEX-S95S-0008-SM-3RQ8B_exp1.FAK55217.bam",
    "GTEX-S95S-0008-SM-3RQ8B_exp2.FAK47088.bam",
    "GTEX-WY7C-0008-SM-3NZB5_ctrl.FAK55679.bam",
    "GTEX-1GN1W-0226-SM-7AGLJ_rep.FAK91654.bam",
    "GTEX-WY7C-1126-SM-3GS2X_rep2.FAK49168.bam",
    "GTEX-WY7C-1126-SM-3GS2X.FAK39149.bam",
    "GTEX-Y5LM-0426-SM-3YX99.FAK52212.bam",
    "GTEX-Y5LM-0426-SM-3YX99_rep2.FAK41279.bam",
    "GTEX-14BMU-0526-SM-5CA2F.FAK44778.bam",
    "GTEX-14BMU-0526-SM-5CA2F_rep.FAK93376.bam",
    "GTEX-13QJ3-0726-SM-7LDHS.FAK49189.bam",
    "GTEX-ZT9X-1826-SM-4V2KV_rep.FAK39773.bam",
    "GTEX-ZT9X-1826-SM-4V2KV.FAK49260.bam", 
    "GTEX-WY7C-0726-SM-3GLGQ.FAK46872.bam" ]


In [12]:
## Load matrix
df = pd.read_csv("../../data/raw/GRCh38_quant_mapq10_gtf_109_and_high-confidence_GTEx_DATA/bambu_quant/counts_transcript.txt", sep="\t")
# load unique counts matrix
df_unique = pd.read_csv("../../data/raw/GRCh38_quant_mapq10_gtf_109_and_high-confidence_GTEx_DATA/bambu_quant/uniqueCounts_transcript.txt", sep="\t")
# load gtex metadata
gtex_metadata = pd.read_csv("/pscratch/mteb223_uksr/new_RNA_isoform_expression_across_tissues/data/GTEx_v9_ONT_metadata.txt", sep = "\t")

## Convert sample names to bam files
columns_to_rename = df.columns[2:]
new_col_names = {col: get_samp_id(col) for col in columns_to_rename}
df.rename(columns=new_col_names, inplace=True)
df = df[df.columns.difference(filter_out_samples)]
df_unique.rename(columns=new_col_names, inplace=True)
df_unique = df_unique[df_unique.columns.difference(filter_out_samples)]

# get the list of samples so that we can separate out the columns below by tissue
df_cols_dict = {}
for tissue in gtex_metadata["tissue_site_detail"].copy().unique():
    for tissue in tissues_to_use:
        df_cols_dict[tissue] = [el for el in gtex_metadata.loc[gtex_metadata['tissue_site_detail'] == tissue, 'bam_file'].values.tolist() if not 'direct' in el]
        df_cols_dict[tissue] = [x for x in df_cols_dict[tissue] if x not in filter_out_samples]

df_by_tissue = {}
#print(df.head())
for key in df_cols_dict:

    # grab the columns we will need for this tissue
    columns = ["TXNAME", "GENEID"] + df_cols_dict[key]
    tmp = df[columns].copy()
    
    ## Fix columns names
    tmp = fix_column_names(tmp, is_gene=False)
    
    ## Add total counts column
    tmp["total_counts"] = tmp[tmp.filter(regex='count').columns].copy().sum(axis=1)
    
    ## CPM normalize counts matrix
    tmp = calculate_cpm(tmp, is_gene=False)
    tmp["median_CPM"] = round(tmp[tmp.filter(regex='CPM').columns].copy().median(axis=1), 2)
    tmp.drop(columns=tmp.filter(regex='count').columns, inplace=True)
    
    tmp_u = df_unique[columns].copy()
    
    ## Fix columns names
    tmp_u = fix_column_names(tmp_u, is_gene=False)
    ## Add total counts column
    tmp_u["median_unique_counts"] = tmp_u[tmp_u.filter(regex='count').columns].copy().median(axis=1)
    # for the unique dataframe, keep only these columns for merging
    tmp_u = tmp_u[['transcript_id', 'gene_id', 'median_unique_counts']]

    # merge the all counts and unique counts together
    tmp = pd.merge(tmp, tmp_u, how='inner')
    # calculate the relative transcript abundance
    tmp = relative_transcript_abundance(tmp.copy())
    # add the tissue name
    tmp['tissue_site_detail'] = key
    # add on the biotype of the transcript and gene
    tmp = tmp.merge(transcript_biotype, how='left')
    tmp = tmp.merge(gene_info, how='left')
    # write table to file
    tmp.to_csv("../../data/processed/GTEx/GTEX_since2019_" + key + "_cpm_transcript.txt", sep="\t", index=False)
    n_filtered_out = len(tmp.loc[tmp["median_unique_counts"] < 1])
    # filter -> transcripts must be present with at least one unique count in at least half the samples
    tmp = tmp.loc[tmp["median_unique_counts"] >= 1].copy()
    df_by_tissue[key] = tmp
    


In [13]:
df_by_tissue['Lung'].head()

,transcript_id,gene_id,GTEX-1211K-0826-SM-7LDFQ.FAK46515.bam_CPM,GTEX-14BMU-0526-SM-5CA2F_rep2.FAK49039.bam_CPM,GTEX-1I6K7-1226-SM-AAEQX.FAK44642.bam_CPM,GTEX-1KXAM-0426-SM-CYKMP.FAK44752.bam_CPM,GTEX-WYVS-0526-SM-3H5V7.FAK54827.bam_CPM,GTEX-ZT9X-0326-SM-4U9QG.FAK44894.bam_CPM,total_CPM,median_CPM,median_unique_counts,median_relative_abundance,median_gene_cpm,tissue_site_detail,transcript_biotype,gene_biotype,gene_name
0,BambuTx1,ENSG00000227232,0.45,1.67,3.33,2.07,0.56,1.31,1.56,1.56,6.5,73.231896,2.600,Lung,NaN,unprocessed_pseudogene,WASH7P
1,ENST00000488147,ENSG00000227232,0.00,0.42,0.56,1.04,0.37,1.83,0.72,0.56,2.0,26.768104,2.600,Lung,unprocessed_pseudogene,unprocessed_pseudogene,WASH7P
2,BambuTx100,ENSG00000215861,3.30,0.00,1.96,3.19,2.72,3.22,2.27,2.72,9.5,15.137348,19.685,Lung,NaN,transcribed_unprocessed_pseudogene,NaN
4,BambuTx97,ENSG00000215861,11.94,21.86,22.99,22.50,9.86,12.88,17.01,17.01,41.5,79.024450,19.685,Lung,NaN,transcribed_unprocessed_pseudogene,NaN
8,BambuTx1006,ENSG00000125611,0.00,0.84,0.28,0.26,0.19,0.78,0.42,0.28,1.0,0.324856,86.400,Lung,NaN,protein_coding,CHCHD5


In [14]:
# verify the number of samples being used from each tissue
samps_we_use = []
total_samples = 0
for tiss in df_cols_dict:
    # print the tissue, the sample ids, and the number of samples
    print(tiss)
    print(df_cols_dict[tiss])
    samps_we_use = samps_we_use + df_cols_dict[tiss]
    print(len(df_cols_dict[tiss]))
    total_samples = total_samples + len(df_cols_dict[tiss])
print(total_samples)

Brain - Cerebellar Hemisphere
['GTEX-11H98-0011-R11b-SM-4SFLZ.FAK46829.bam', 'GTEX-13VXU-0011-R11b-SM-5BFQZ.FAK44611.bam', 'GTEX-17F97-0011-R11b-SM-63KY2.FAK41775.bam', 'GTEX-1H3NZ-0011-R11b-SM-AUNOV.FAK49024.bam', 'GTEX-Q2AG-0011-R11A-SM-2EBL2.FAK42265.bam', 'GTEX-T5JC-0011-R11A-SM-2TT24.FAK54838.bam']
6
Brain - Frontal Cortex (BA9)
['GTEX-1192X-0011-R10a-SM-4RXXZ.FAK49046.bam', 'GTEX-13X6J-0011-R10b-SM-5CEKT.FAK44896.bam', 'GTEX-14BIL-0011-R10a-SM-5EQV4.FAK49209.bam', 'GTEX-15DCD-0011-R10b-SM-5S51M.FAK42101.bam', 'GTEX-QDT8-0011-R10A-SM-2FKJB.FAK49182.bam']
5
Brain - Putamen (basal ganglia)
['GTEX-11TTK-0011-R7b-SM-4TVFS.FAK39197.bam', 'GTEX-1313W-0011-R7b-SM-4ZL3U.FAK44754.bam', 'GTEX-13RTJ-0011-R7b-SM-5CTCB.FAK49257.bam', 'GTEX-14C5O-0011-R7b-SM-5GUPO.FAK54887.bam', 'GTEX-15ER7-0011-R7a-SM-5QYP2.FAK44704.bam', 'GTEX-T5JC-0011-R7A-SM-2TT1Z.FAK42170.bam']
6
Cells - Cultured fibroblasts
['GTEX-OIZI-0008-SM-2FR3P.FAK41829.bam', 'GTEX-OXRL-0008-SM-2FR3T.FAK39703.bam', 'GTEX-PSDG-0008-SM

## Make figures looking at brain level expression of new isoforms since 2019

In [15]:
## Only keep transcripts that are exclusive to the 2023 annotation when compared to 2019
exclusive_2023_transcripts = transcript_ref_2023.loc[~transcript_ref_2023["transcript_id"].isin(transcript_ref_2019_ids)].copy()

In [16]:
## Create list of new CCDS since 2019

## Get all transcript IDs from ENSEMBL 96 GTF (2019) and their protein_id that we generated
cds_2019_ref = transcript_ref_2019.merge(ref_cds, how="inner", on="transcript_id").dropna(subset="protein_id")
cds_2019_ref_protein_ids = cds_2019_ref["protein_id"].copy()

## Get transcript names from ENSEMBL 109 GTF (2023)
cds_2023_ref = transcript_ref_2023.merge(ref_cds, how="inner", on="transcript_id").dropna(subset="protein_id")

## Get ensmbl 109 (2023) exclusive protein coding sequences
exclusive_2023_cds = cds_2023_ref.loc[~cds_2023_ref["transcript_id"].isin(transcript_ref_2019_ids)].copy()
exclusive_2023_cds = exclusive_2023_cds.loc[~exclusive_2023_cds["protein_id"].isin(cds_2019_ref_protein_ids)].copy()

In [17]:
# initialize variables
df_exclusive_2023_transcripts = {}
df_exclusive_2023_ccds = {}

list_cpm_thresh = [x/100 for x in range(0,1002)]

## Create lists with numbers of transcripts expressed across CPM thresholds for different categories
list_2023_all_transcript_median = {}
list_2023_all_gene_median = {}
list_2023_new_cds_transcript_median = {}
list_2023_med_relevant_transcript_median = {}
list_2023_med_relevant_new_cds_transcript_median = {}
list_2023_brain_relevant_transcript_median = {}
list_2023_brain_relevant_new_cds_transcript_median = {}

exclusive_2023_tx_df = pd.DataFrame()
isoforms_in_categories = {}

values_at_cpm_threshold = pd.DataFrame(columns=['cpm_threshold', 'tissue', 
                                                'all_transcript_median', 
                                                'all_gene_median',
                                                'new_cds_transcript_median', 
                                                'med_relevant_transcript_median', 
                                               'med_relevant_new_cds_transcript_median',
                                               'brain_relevant_transcript_median',
                                               'brain_relevant_new_cds_transcript_median'])

## Create lists with numbers of 2023 exclusive transcripts expressed across CPM thresholds
for tissue in df_by_tissue:
    
    # grab the tissue we are working with
    tmp = df_by_tissue[tissue].copy()
    # filter to only new since 2019 TX
    df_exclusive_2023_transcripts[tissue] = tmp.loc[tmp["transcript_id"].isin(exclusive_2023_transcripts["transcript_id"])].copy()

    prep_for_exclusive_2023_tx_df = df_exclusive_2023_transcripts[tissue].copy()[['gene_id', 'gene_name', 'transcript_id', 'tissue_site_detail', 'median_CPM', 'median_unique_counts', 'median_relative_abundance', 'median_gene_cpm', 'transcript_biotype', 'gene_biotype']]
    exclusive_2023_tx_df = pd.concat([exclusive_2023_tx_df, prep_for_exclusive_2023_tx_df.copy()])
    
    # looking at only those with a new cds region
    df_exclusive_2023_ccds[tissue] = tmp.loc[tmp["transcript_id"].isin(exclusive_2023_cds["transcript_id"])].copy()
    
    # create empty list for the tissue
    list_2023_all_transcript_median[tissue] = []
    list_2023_all_gene_median[tissue] = []
    list_2023_new_cds_transcript_median[tissue] = []
    list_2023_med_relevant_transcript_median[tissue] = []
    list_2023_med_relevant_new_cds_transcript_median[tissue] = []
    list_2023_brain_relevant_transcript_median[tissue] = []
    list_2023_brain_relevant_new_cds_transcript_median[tissue] = []
    
    # for a range of CPM thresholds
    for i in range(0, 1002):

        cpm_thresh = i/100

        # get the isoforms that fit each category
        median_2023 = df_exclusive_2023_transcripts[tissue].loc[df_exclusive_2023_transcripts[tissue]["median_CPM"] >= cpm_thresh].copy()
        new_cds_median_2023 = df_exclusive_2023_ccds[tissue].loc[df_exclusive_2023_ccds[tissue]["median_CPM"] >= cpm_thresh].copy()
        med_relevant_median_2023 = median_2023.loc[median_2023["gene_id"].isin(disease_relevant_genes["gene_id"])].copy()
        med_relevant_new_cds_median_2023 = med_relevant_median_2023.loc[med_relevant_median_2023["transcript_id"].isin(new_cds_median_2023["transcript_id"])].copy()
        brain_relevant_median_2023 = median_2023.loc[median_2023["gene_id"].isin(brain_disease_gene_ids["gene_id"])].copy()
        brain_relevant_new_cds_median_2023 = brain_relevant_median_2023.loc[brain_relevant_median_2023["transcript_id"].isin(new_cds_median_2023["transcript_id"])].copy()
        
        # at CPM > 1 (in this case, because we round values to 2 decimals, CPM == 1.01 is equivalent to CPM > 1)
        if cpm_thresh == 1.01:
            # grab modified dataframes for later analysis
            isoforms_in_categories[tissue] = {
                "All isoforms": median_2023[['gene_id', 'gene_name', 'transcript_id', 'median_relative_abundance']],
                "Med-relevant": med_relevant_median_2023[['gene_id', 'gene_name', 'transcript_id', 'median_relative_abundance']],
                "New protein coding sequence": new_cds_median_2023[['gene_id', 'gene_name', 'transcript_id', 'median_relative_abundance']],
                "Med-relevant new protein coding sequence": med_relevant_new_cds_median_2023[['gene_id', 'gene_name', 'transcript_id', 'median_relative_abundance']],
                "Brain disease relevant": brain_relevant_median_2023[['gene_id', 'gene_name', 'transcript_id', 'median_relative_abundance']],
                "Brain disease relevant new protein coding sequence": brain_relevant_new_cds_median_2023[['gene_id', 'gene_name', 'transcript_id', 'median_relative_abundance']],
            }

        # add just the total number in each category to the list
        list_2023_all_transcript_median[tissue].append(median_2023.shape[0])
        list_2023_all_gene_median[tissue].append(median_2023['gene_id'].nunique())
        list_2023_new_cds_transcript_median[tissue].append(new_cds_median_2023.shape[0])
        list_2023_med_relevant_transcript_median[tissue].append(med_relevant_median_2023.shape[0])
        list_2023_med_relevant_new_cds_transcript_median[tissue].append(med_relevant_new_cds_median_2023.shape[0])
        list_2023_brain_relevant_transcript_median[tissue].append(brain_relevant_median_2023.shape[0])
        list_2023_brain_relevant_new_cds_transcript_median[tissue].append(brain_relevant_new_cds_median_2023.shape[0])
       
    # concat values to dataframe holding all the numbers for the thresholds for all tissues
    values_at_cpm_threshold = pd.concat([values_at_cpm_threshold, pd.DataFrame({
        'cpm_threshold': list_cpm_thresh,
        'tissue': tissue,
        'all_transcript_median': list_2023_all_transcript_median[tissue],
        'all_gene_median': list_2023_all_gene_median[tissue],
        'new_cds_transcript_median': list_2023_new_cds_transcript_median[tissue],
        'med_relevant_transcript_median': list_2023_med_relevant_transcript_median[tissue],
        'med_relevant_new_cds_transcript_median': list_2023_med_relevant_new_cds_transcript_median[tissue],
        'brain_relevant_transcript_median': list_2023_brain_relevant_transcript_median[tissue],
        'brain_relevant_new_cds_transcript_median': list_2023_brain_relevant_new_cds_transcript_median[tissue]
    })])


# write to files
values_at_cpm_threshold.to_csv("../../tables/GTEx_expression_since_2019/gtex_values_at_cpm_thresholds_new_since_2019.tsv", sep = '\t', index=False)
exclusive_2023_tx_df.to_csv('../../tables/GTEx_expression_since_2019/gtex_values_cpm_tx_new_since_2019.tsv', sep='\t', index=False)

## Run venn comparisons between tissues

In [18]:
# compare two brain regions and lung
lung_br = set(isoforms_in_categories['Lung']['Brain disease relevant']['transcript_id'].tolist())
cereb_br = set(isoforms_in_categories['Brain - Cerebellar Hemisphere']['Brain disease relevant']['transcript_id'].tolist())
front_br = set(isoforms_in_categories['Brain - Frontal Cortex (BA9)']['Brain disease relevant']['transcript_id'].tolist())

# Get the full intersect of 'transcript_id' from all DataFrames
full_intersect = lung_br & cereb_br & front_br

# Get the intersect of 'transcript_id' between two DataFrames
lung_and_cereb = lung_br & cereb_br
cereb_and_front = cereb_br & front_br
front_and_lung = front_br & lung_br

# Calculate the lengths for each set operation
overlap_lengths = {
    'lung,cereb,front': len(full_intersect),
    'lung,cereb': len(lung_and_cereb - full_intersect),
    'cereb,front': len(cereb_and_front - full_intersect),
    'front,lung': len(front_and_lung - full_intersect),
    'lung': len(lung_br - (lung_and_cereb | front_and_lung)),
    'cereb': len(cereb_br - (lung_and_cereb | cereb_and_front)),
    'front': len(front_br - (cereb_and_front | front_and_lung))
}

print(overlap_lengths)
print("ALL:")
print(full_intersect)
print("just lung")
print(lung_br - (lung_and_cereb | front_and_lung))

{'lung,cereb,front': 8, 'lung,cereb': 3, 'cereb,front': 7, 'front,lung': 3, 'lung': 20, 'cereb': 19, 'front': 5}
ALL:
{'ENST00000678921', 'ENST00000676712', 'ENST00000677047', 'ENST00000679902', 'ENST00000696840', 'ENST00000707132', 'ENST00000678242', 'ENST00000682710'}
just lung
{'ENST00000676865', 'ENST00000680422', 'ENST00000677976', 'ENST00000676502', 'ENST00000679172', 'ENST00000678615', 'ENST00000678727', 'ENST00000677207', 'ENST00000698131', 'ENST00000699277', 'ENST00000678886', 'ENST00000679214', 'ENST00000677418', 'ENST00000678031', 'ENST00000679224', 'ENST00000677544', 'ENST00000678929', 'ENST00000679887', 'ENST00000679051', 'ENST00000678114'}


In [19]:
# look at the intersection of all 
put_br = set(isoforms_in_categories['Brain - Putamen (basal ganglia)']['Brain disease relevant']['transcript_id'].tolist())
cereb_br = set(isoforms_in_categories['Brain - Cerebellar Hemisphere']['Brain disease relevant']['transcript_id'].tolist())
front_br = set(isoforms_in_categories['Brain - Frontal Cortex (BA9)']['Brain disease relevant']['transcript_id'].tolist())
cells_br = set(isoforms_in_categories['Cells - Cultured fibroblasts']['Brain disease relevant']['transcript_id'].tolist())
haa_br = set(isoforms_in_categories['Heart - Atrial Appendage']['Brain disease relevant']['transcript_id'].tolist())
hlv_br = set(isoforms_in_categories['Heart - Left Ventricle']['Brain disease relevant']['transcript_id'].tolist())
lung_br = set(isoforms_in_categories['Lung']['Brain disease relevant']['transcript_id'].tolist())
liv_br = set(isoforms_in_categories['Liver']['Brain disease relevant']['transcript_id'].tolist())
mus_br = set(isoforms_in_categories['Muscle - Skeletal']['Brain disease relevant']['transcript_id'].tolist())

print(put_br & cereb_br & front_br & cells_br & haa_br & hlv_br & lung_br & liv_br & mus_br)

{'ENST00000678921', 'ENST00000678242', 'ENST00000679902'}


In [20]:
# look at the intersection of brain regions
print(put_br & cereb_br & front_br)

{'ENST00000678921', 'ENST00000674619', 'ENST00000696840', 'ENST00000679902', 'ENST00000707132', 'ENST00000678242', 'ENST00000675737', 'ENST00000676437'}


In [21]:
# compare lung, cerebellum, and cultured fibroblasts
lung_br = set(isoforms_in_categories['Lung']['Brain disease relevant']['transcript_id'].tolist())
cereb_br = set(isoforms_in_categories['Brain - Cerebellar Hemisphere']['Brain disease relevant']['transcript_id'].tolist())
cell_br = set(isoforms_in_categories['Cells - Cultured fibroblasts']['Brain disease relevant']['transcript_id'].tolist())

# Get the full intersect of 'transcript_id' from all DataFrames
full_intersect = lung_br & cereb_br & front_br

# Get the intersect of 'transcript_id' between two DataFrames
lung_and_cereb = lung_br & cereb_br
cereb_and_cell = cereb_br & cell_br
cell_and_lung = cell_br & lung_br

# Calculate the lengths for each set operation
overlap_lengths = {
    'lung,cereb,cells': len(full_intersect),
    'lung,cereb': len(lung_and_cereb - full_intersect),
    'cereb,cells': len(cereb_and_cell - full_intersect),
    'cells,lung': len(cell_and_lung - full_intersect),
    'lung': len(lung_br - (lung_and_cereb | cell_and_lung)),
    'cereb': len(cereb_br - (lung_and_cereb | cereb_and_cell)),
    'cells': len(cell_br - (cereb_and_cell | cell_and_lung))
}

print(overlap_lengths)
print("ALL:")
print(full_intersect)
print('just lung:')
print(lung_br - (lung_and_cereb | cell_and_lung))

{'lung,cereb,cells': 8, 'lung,cereb': 3, 'cereb,cells': 1, 'cells,lung': 9, 'lung': 14, 'cereb': 25, 'cells': 6}
ALL:
{'ENST00000678921', 'ENST00000676712', 'ENST00000677047', 'ENST00000679902', 'ENST00000696840', 'ENST00000707132', 'ENST00000678242', 'ENST00000682710'}
just lung:
{'ENST00000676865', 'ENST00000680422', 'ENST00000677976', 'ENST00000698131', 'ENST00000679172', 'ENST00000699277', 'ENST00000678886', 'ENST00000678114', 'ENST00000678727', 'ENST00000678031', 'ENST00000679224', 'ENST00000676808', 'ENST00000677207', 'ENST00000679887'}


## Look at relative expression of new isoforms 

In [22]:
## Create dataframe to store percent expression
percent_expression_df = pd.DataFrame()

In [23]:
# gene/isoform categories
cat_to_df = {"Med-relevant", "New protein coding sequence", "Med-relevant new protein coding sequence", "Brain disease relevant", "Brain disease relevant new protein coding sequence"}


for tissue in tissues_to_use:
    for category in cat_to_df:
        ## Get percent expression from transcripts since 2019 for each subcategory
        df_tmp = isoforms_in_categories[tissue][category].copy()
        df_tmp_comb_rel_exp = df_tmp.groupby(['gene_id', 'gene_name'])['median_relative_abundance'].sum().reset_index()
        df_tmp_comb_rel_exp.rename(columns={'median_relative_abundance': 'Relative Abundance of New Transcripts Since 2019 (%)'}, inplace=True)
        df_tmp_comb_rel_exp['Category'] = category
        df_tmp_comb_rel_exp['Tissue'] = tissue
        percent_expression_df = pd.concat([percent_expression_df, df_tmp_comb_rel_exp])

In [24]:
# write to file to use in plots
percent_expression_df.to_csv("../../tables/GTEx_expression_since_2019/gtex_cpm_gt_1_relative_abundance_new_since_2019.tsv", sep = '\t', index=False)
percent_expression_df.head()

,gene_id,gene_name,Relative Abundance of New Transcripts Since 2019 (%),Category,Tissue
0,ENSG00000001497,LAS1L,3.420882,Med-relevant new protein coding sequence,Brain - Cerebellar Hemisphere
1,ENSG00000003393,ALS2,3.611826,Med-relevant new protein coding sequence,Brain - Cerebellar Hemisphere
2,ENSG00000004455,AK2,48.969782,Med-relevant new protein coding sequence,Brain - Cerebellar Hemisphere
3,ENSG00000008441,NFIX,0.398749,Med-relevant new protein coding sequence,Brain - Cerebellar Hemisphere
4,ENSG00000021645,NRXN3,0.511729,Med-relevant new protein coding sequence,Brain - Cerebellar Hemisphere
